In [38]:
import ms4_franklab_pyplines as pyp
import pyff_utils as pyff
import ms4_franklab_proc2py as p2p
import sys, os, subprocess
import logging
from distutils.dir_util import copy_tree
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%autoreload 2

In [40]:
# anim details. for now, assume that sort should include all eps/day
mda_util_path = '/home/ag/SRC/frank-lab/mda_util.py'
ntlinks_path = '/home/ag/SRC/frank-lab/MS4setup_NTlinks.node.js'
data_dir = '/home/ag/SRC/frank-lab/'
results_dir= '/home/ag/SRC/frank-lab/results/'
anim = 'G02'
days = [1]
dates = [20180609] #dates 10/22-11/30

# if datelist not provided, fetch dates from taskinfo struct
# TODO make py_evaluate filter more flexible so that it can do this

#if not dates or len(days)!=len(dates):
#    dates = pyff.py_evaluatefilter(base_dir, anim, 'taskinfo', days,['date','*'])

#if tetlist not provided, fetch ca1 tets from tetinfo struct
print('I ran! 1')
tetfilt = ['area','ca1']
tetlist = [1, 2]
if not tetlist and tetfilt:
    tetlist = pyff.py_evaluatefilter(results_dir, anim, 'tetinfo', days,tetfilt)

I ran! 1


In [41]:
days

[1]

In [58]:
    index = 0
    day = 1
    day_src_dir = data_dir+anim+'/preprocessing/'+str(dates[index])+'/'
    # check if mnt directory exists for this day. if not, call mda_util to generate the symlinks to the mda files
    # run this in the data src directory (vortex)
    mnt_path = day_src_dir+str(dates[index])+'_'+anim+'.mnt/'
    print('processing day '+str(day))
    if not os.path.exists(mnt_path):
        print('no mnt directory found; calling mda_util')
        os.chdir(data_dir+anim+'/preprocessing')
        subprocess.call(['python',mda_util_path])
    print('MDA Util Ran successfully')

processing day 1
no mnt directory found; calling mda_util
MDA Util Ran successfully


In [57]:
    mountain_src_path = day_src_dir+str(dates[index])+'_'+anim+'.mountain'
    mountain_res_path = results_dir+anim+'/preprocessing/'+str(dates[index])+'/'+str(dates[index])+'_'+anim+'.mountain'
    
    print('Source ' + mountain_src_path)
    print('Destination ' + mountain_res_path)
    if not os.path.exists(mountain_src_path):
        print('no mountain dir found; calling setup_NTlinks')
        os.chdir(day_src_dir)
        subprocess.call([ntlinks_path, mnt_path])
    print('Finished creating NT Links')
    if not os.path.exists(mountain_res_path):
        print('copying mountain dir to results dir')
        copy_tree(mountain_src_path,mountain_res_path)
    else:
        print(mountain_res_path + ' exists')

Source /home/ag/SRC/frank-lab/G02/preprocessing/20180609/20180609_G02.mountain
Destination /home/ag/SRC/frank-lab/results/G02/preprocessing/20180609/20180609_G02.mountain
no mountain dir found; calling setup_NTlinks
Finished creating NT Links
copying mountain dir to results dir


DistutilsFileError: cannot copy tree '/home/ag/SRC/frank-lab/G02/preprocessing/20180609/20180609_G02.mountain': not a directory

In [31]:
        nt_ind = 0
        nt = 2
        nt_src_dir = mountain_src_path+'/nt'+str(nt)
        nt_out_dir = mountain_res_path+'/nt'+str(nt)
        
        # concatenate all eps, since ms4 no longer takes a list of mdas; save as raw.mda
        # save this to the output dir; it serves as src for subsequent steps
        prv_list=nt_src_dir+'/raw.mda.prv'
        print('Calling concat_eps')
        print()
        pyp.concat_eps(dataset_dir=nt_out_dir,mda_list=prv_list)
        
        # preprocessing: filter, mask out artifacts, whiten
        # TODO make optional whether you save the interediates (filt, pre)
        pyp.filt_mask_whiten(dataset_dir=nt_out_dir,output_dir=nt_out_dir, freq_min=300,freq_max=6000, opts={})
        
        #run the actual sort 
        pyp.ms4_sort_on_segs(dataset_dir=nt_out_dir,output_dir=nt_out_dir, adjacency_radius=-1,detect_threshold=3, detect_sign=-1, opts={})
        
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.add_curation_tags(dataset_dir=nt_dir,output_dir=nt_dir,opts={})
        pyp.extract_marks(dataset_dir=nt_dir,output_dir=nt_dir,opts={})

        

Calling concat_eps



FileNotFoundError: [Errno 2] No such file or directory: '/home/ag/SRC/frank-lab/G02/preprocessing/20180609/20180609_G02.mountain/nt2/raw.mda.prv'

In [1]:
for index, day in enumerate(days):

    day_src_dir = data_dir+anim+'/preprocessing/'+str(dates[index])+'/'
    # check if mnt directory exists for this day. if not, call mda_util to generate the symlinks to the mda files
    # run this in the data src directory (vortex)
    mnt_path = day_src_dir+str(dates[index])+'_'+anim+'.mnt/'
    print('processing day '+str(day))
    if not os.path.exists(mnt_path):
        print('no mnt directory found; calling mda_util')
        os.chdir(data_dir+anim+'/preprocessing')
        subprocess.call(['python',mda_util_path])  
    
    # check if .mountain dir exists for this day (still in the data src  dir)
    # if not, run the setup javascript to generate prvs that link mda files (via their symlinks) across epochs
    # need to generate mountaindir in src then copy it over to results
    mountain_src_path = cle+str(dates[index])+'_'+anim+'.mountain'
    mountain_res_path = results_dir+anim+'/preprocessing/'+str(dates[index])+'/'+str(dates[index])+'_'+anim+'.mountain'
    if not os.path.exists(mountain_src_path):
        print('no mountain dir found; calling setup_NTlinks')
        os.chdir(day_src_dir)
        subprocess.call([ntlinks_path, mnt_path])
    if not os.path.exists(mountain_res_path):
        print('copying mountain dir to results dir')
        copy_tree(mountain_src_path,mountain_res_path)
    
    #now for the actually processing: iterate through ntrodes
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_src_dir = mountain_src_path+'/nt'+str(nt)
        nt_out_dir = mountain_res_path+'/nt'+str(nt)
        
        # concatenate all eps, since ms4 no longer takes a list of mdas; save as raw.mda
        # save this to the output dir; it serves as src for subsequent steps
        prv_list=nt_src_dir+'/raw.mda.prv'
        pyp.concat_eps(dataset_dir=nt_out_dir,mda_list=prv_list)
        
        # preprocessing: filter, mask out artifacts, whiten
        # TODO make optional whether you save the interediates (filt, pre)
        pyp.filt_mask_whiten(dataset_dir=nt_out_dir,output_dir=nt_out_dir, freq_min=300,freq_max=6000, opts={})
        
        #run the actual sort 
        pyp.ms4_sort_on_segs(dataset_dir=nt_out_dir,output_dir=nt_out_dir, adjacency_radius=-1,detect_threshold=3, detect_sign=-1, opts={})
        
        

NameError: name 'days' is not defined

In [ ]:
# in a separate loop, go back and add tags to the metrics - due to bug in ms3.combine_cluster_metrics
for index, day in enumerate(days):
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.add_curation_tags(dataset_dir=nt_dir,output_dir=nt_dir,opts={})


In [ ]:
# generate marks 
for index, day in enumerate(days):
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.extract_marks(dataset_dir=nt_dir,output_dir=nt_dir,opts={})

In [ ]:
# TODO: implement logging system to track all calls and params used 